<a href="https://colab.research.google.com/github/kaushikabhishek87/Tensorflow_projects/blob/main/04_Transfer_Learning_Tensorflow_part_1_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Trasnfer learning with Tensorflow - Feature Extraction